<a href="https://colab.research.google.com/github/Gutomoby/BolsaBrasileira/blob/main/FundsExplorer_Ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Funds Explorer Rank - Crawler

## Objetivo
Este notebook tem por objetivo facilitar a seleção de fundos de investimentos imobiliários(FIIs), para constiutição de carteira previdênciaria com bons distribuidores de proventos, cuja distribuição seja sustentável a longo prazo.

## Método
- Extração de dados do site [Funds Explorer](https://www.fundsexplorer.com.br/)
- Tratamento dos dados
- Filtro baseado em requisitos minimos de aceitação
- É necessária análise qualitativa dos papéis resultantes

In [ ]:
import datetime
import logging
import requests
import numpy as np
import pandas as pd
import pytz

## Extração dos dados (Data Extraction)

In [ ]:
def now():
    """
    Current timestamp at time zone utc
    :returns: timestamp
    """
    return pytz.UTC.localize(datetime.datetime.utcnow())


def now_br():
    """
    Current timestamp at time zone América São Paulo
    :returns: timestamp
    """
    return now().astimezone(pytz.timezone("America/Sao_Paulo"))

In [ ]:
# Logging basic configuration
logging.basicConfig(level=logging.INFO)
_logger = logging.getLogger()

In [ ]:
# Updated at
str(now_br())

'2021-05-25 15:17:55.995607-03:00'

In [ ]:
url = 'https://www.fundsexplorer.com.br/ranking'
headers = {
    'User-Agent': 
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'
        ' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
}

# Collecting Data
response = requests.get(url, headers)
_logger.info("%s url request's status: %s", url, response.status_code)
# List object of Data Frames
list_obj = pd.read_html(response.text, attrs={'id': 'table-ranking'})
df = list_obj[0]

INFO:root:https://www.fundsexplorer.com.br/ranking url request's status: 200
INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df.dtypes

## Limpeza de dados (Data Cleaning)

### Cabeçalhos
- Remoção de espaços
- Remoção de acentos e pontuação
- Transformação dos caracteres para minúsculo

In [ ]:
# Cleanning data headers
df.columns = df.columns.str.replace('\s+', '_')
df.columns = df.columns.str.replace(r'[^\w\s]+', '_')
df.columns = df.columns.str.lower()
df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df.columns

### Valores
- Conversão de moeda em formatode texto para número
- Remoção de espaços
- Remoção de acentos e pontuação
- Transformação dos caracteres para minúsculo


In [ ]:
df.head()

In [ ]:
def columns_which_contains(df, value):
    """
    Serch for DataFrame column's values which contains a specific value
    :value: any characters, phrases, symbols
    :returns: list of DataFrame column's names
    """
    return [df[column].name for column in df if df[column].astype(str).str.contains(value).any()]

In [ ]:
# Convert string BR currency to float
currency_columns = columns_which_contains(df, '\$')
_logger.info("Currency Columns: %s", currency_columns)
for column_name in currency_columns:
    df[column_name] = df[column_name].str.replace('.', '')
    df[column_name] = df[column_name].str.replace(',', '.')
    df[column_name] = df[column_name].str.replace('[R\$ ,]', '', regex=True).astype(float)

In [ ]:
# Convert string % to float
percentual_columns = columns_which_contains(df, '\%')
_logger.info("Percentual Columns: %s", percentual_columns)
for column_name in percentual_columns:
    df[column_name] = df[column_name].str.replace(',', '.')
    df[column_name] = df[column_name].str.replace('[%,]', '', regex=True).astype(float)
    df[column_name] = df[column_name]/100

In [ ]:
df.head()

In [ ]:
df['setor'] = df['setor'].str.replace('\s+', '_')
df['setor'] = df['setor'].str.replace(r'[^\w\s]+', '_')
df['setor'] = df['setor'].str.lower()
df['setor'] = df['setor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

## Definindo requisitos para filtro

In [ ]:
# Convert p/vpa to the correct base
df['p_vpa'] = df['p_vpa']/100

# Price vs net worth

col         = 'p_vpa'
conditions  = [ (df[col] > 0.9) & (df[col] < 1.1), (df[col] <= 0.9), df[col] >= 1.1, df[col].isnull() ]
choices     = [ 'Mercado', 'Desconto', 'Caro', 'Sem histórico' ]

df['desconto_mercado'] = np.select(conditions, choices, default=np.nan)

# Asset diversification
col         = 'quantidadeativos'
conditions  = [ df[col] > 7, df[col] < 1, (df[col] >= 1) & (df[col] < 7) ]
choices     = [ 'Diversificado', 'Não Possui', 'Concentrado' ]
df['diversidade_de_ativos'] = np.select(conditions, choices, default=np.nan)

# Vacancy

col         = 'vacanciafisica'
conditions  = [ (df[col] > 0.1) & (df[col] < 0.15), df[col] > 0.15, df[col] < 0.1, df[col].isnull() ]
choices     = [ 'Média', 'Alta', 'Baixa', 'Sem histórico' ]

df['vacância_comparativo'] = np.select(conditions, choices, default=np.nan)

# Liquidity

col         = 'liquidez_diaria'
conditions  = [ (df[col] > 0) & (df[col] < 25000), (df[col] > 25000) & (df[col] < 100000), df[col] > 1000000, df[col].isnull() ]
choices     = [ 'Ilíquido', 'Média Liquidez', 'Alta Liquidez', 'Sem histórico' ]

df['volume_negociado'] = np.select(conditions, choices, default=np.nan)

# dividend yield

col         = 'dividendyield'
conditions  = [ (df[col] > 0.006) & (df[col] < 0.008), (df[col] <= 0.006), df[col] >= 0.008, df[col].isnull() ]
choices     = [ 'Yield Padrão', 'Yield Baixo', 'Yield Alto', 'Sem histórico' ]

df['distribuição'] = np.select(conditions, choices, default=np.nan)

### Resultado
*Os papéis abaixo não são recomendação de compra, mas sim estudo*.

In [ ]:
pd.options.display.max_rows = 999
resultado = df.drop(['dypatrimonial','variacaopatrimonial','rentab__patr_no_periodo','rentab__patr_acumulada','vacanciafinanceira'], axis=1)
resultado.to_excel("analise_quant_automatica.xlsx", sheet_name='Analise_Robo')